In [1]:
# Dependencies
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

In [2]:
types_df = pd.read_csv("../Resources/ethnic_restr.csv")
types_df.head()

,ethnicity
0,mexican
1,italian
2,chinese
3,french
4,thai


In [3]:
# set up additional columns to hold information
types_df['name'] = ""
types_df['address'] = ""
types_df['price_level'] = ""
types_df['rating'] = ""

types_df.head()

,ethnicity,name,address,price_level,rating
0,mexican,,,,
1,italian,,,,
2,chinese,,,,
3,french,,,,
4,thai,,,,


In [10]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: mexican.
Closest mexican restaurant is Chipotle Mexican Grill.
------------
Retrieving Results for Index 1: italian.
Closest italian restaurant is Lascala's.
------------
Retrieving Results for Index 2: chinese.
Closest chinese restaurant is Chinese Fast Wok.
------------
Retrieving Results for Index 3: french.
Closest french restaurant is Del Frisco's Double Eagle Steakhouse.
------------
Retrieving Results for Index 4: thai.
Closest thai restaurant is Chatayee Thai.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: japanese.
Missing field/result... skipping.
------------


In [11]:
types_df

,ethnicity,name,address,price_level,rating
0,mexican,Chipotle Mexican Grill,"1625 Chestnut St Ste F6, Philadelphia",1,3.8
1,italian,Lascala's,"1701 John F Kennedy Blvd, Philadelphia",2,3.5
2,chinese,Chinese Fast Wok,"1500 John F Kennedy Blvd, Philadelphia",2,2.3
3,french,Del Frisco's Double Eagle Steakhouse,"1428-1432 Chestnut St, Philadelphia",4,4.5
4,thai,Chatayee Thai,"1227 Walnut St, Philadelphia",,
5,japanese,Ninja Bao,"1500 JFK Blvd Two Penn Center, underground whi...",2,4.8
